In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/can-i-be-an-influencer/sample_submission.csv
/kaggle/input/can-i-be-an-influencer/test_colors.csv
/kaggle/input/can-i-be-an-influencer/test_keywords.csv
/kaggle/input/can-i-be-an-influencer/train.csv
/kaggle/input/can-i-be-an-influencer/train_colors.csv
/kaggle/input/can-i-be-an-influencer/test.csv
/kaggle/input/can-i-be-an-influencer/train_keywords.csv


In [2]:
df=pd.read_csv("/kaggle/input/can-i-be-an-influencer/train.csv")

In [3]:
df.drop(['photo_url','photo_image_url','photo_submitted_at','height','obs_day','obs_hour','obs_min','obs_sec','width','height','description1','description2','latitude','longitude','country','city','camera_model'],axis=1,inplace=True)

In [4]:
df

,image_id,image_category,aspect_ratio,camera_make,iso,aperture_value,focal_length,exposure_time,total_days,stats_downloads
0,68105c019b,Featured,1.50,Canon,200.0,14.0,50.0,1/60,2361,167503
1,06d11c4edd,Featured,0.67,SONY,100.0,2.5,30.0,1/1250,1255,1839
2,f6599edba1,Featured,0.67,NIKON CORPORATION,64.0,2.8,48.0,1/1600,1138,787
3,0f4bcc2d36,Featured,1.78,DJI,139.0,2.2,4.7,1/100,1579,2049
4,74db502ed5,Featured,1.50,FUJIFILM,400.0,6.4,22.2,1/420,1050,2982
...,...,...,...,...,...,...,...,...,...,...
12192,18eaca644e,Featured,0.73,Canon,100.0,5.6,300.0,1/400,1265,2353
12193,eb25cd769e,Featured,0.72,Canon,400.0,5.0,100.0,1/320,1503,3541
12194,73302461d0,Featured,1.50,Canon,400.0,4.0,21.0,1/4000,2501,14581
12195,f2587b2b08,Featured,1.50,NIKON CORPORATION,800.0,1.8,50.0,1/320,2508,9247


In [5]:
def floater(data):
    for i in str(data["exposure_time"]):
        if i.isalpha():
            return None
    spl = str(data["exposure_time"]).split("/")
    
    if len(spl) > 1:
        return float(spl[0])/float(spl[1])
    else:
        return float(spl[0])

df["exposure_time"] = df.apply(floater, axis=1)

In [6]:
df = df[df["exposure_time"].notna()]
df = df[df["iso"].notna()]
df = df[df["camera_make"].notna()]
df = df[df["aperture_value"].notna()]

In [7]:
df['camera_make']=df['camera_make'].str.upper()
def split_cam():
    for i in range(len(df)):
        df['camera_make'].iloc[i]= (df['camera_make'].iloc[i]).replace(',','').split(' ')[0]

split_cam()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
camera=df['camera_make'].value_counts()[:12].index.tolist()

In [9]:
def cameras(x,camera):
    if  x in camera:
        return x
    else:
        return 'OTHERS'
    
df['camera_make_mod']=df['camera_make'].apply(cameras,args=[camera])

In [10]:
df["camera_make_mod"].value_counts()

CANON         4833
NIKON         2630
SONY          2047
FUJIFILM       853
DJI            444
APPLE          317
PANASONIC      262
OTHERS         176
OLYMPUS        157
HASSELBLAD     146
RICOH          113
SAMSUNG         51
GOPRO           38
Name: camera_make_mod, dtype: int64

In [11]:
df = df.drop(["camera_make"],axis=1)

In [13]:
def onehot(df,col):
    #Performing One Hot Encoding
    cluster_dummies = pd.get_dummies(df[col], prefix=col)

    # Concatenate the original dataframe with the one-hot encoded dataframe
    df = pd.concat([df, cluster_dummies], axis=1)
    df = df.drop([col],axis=1)
    return df

In [14]:
from sklearn.preprocessing import LabelEncoder
def labelencoding(df,col):
    # create an instance of LabelEncoder
    le = LabelEncoder()

    # fit the encoder on the column of your dataframe
    le.fit(df[col])

    # transform the column into encoded integers
    df[col+"_mod"] = le.transform(df[col])
    df[col]=df[col+"_mod"]
    df = df.drop([col+"_mod"],axis=1)
    return df

In [15]:
df=labelencoding(df,"image_category")

In [17]:
df=onehot(df,"camera_make_mod")

In [18]:
df

,image_id,image_category,aspect_ratio,iso,aperture_value,focal_length,exposure_time,total_days,stats_downloads,camera_make_mod_APPLE,...,camera_make_mod_FUJIFILM,camera_make_mod_GOPRO,camera_make_mod_HASSELBLAD,camera_make_mod_NIKON,camera_make_mod_OLYMPUS,camera_make_mod_OTHERS,camera_make_mod_PANASONIC,camera_make_mod_RICOH,camera_make_mod_SAMSUNG,camera_make_mod_SONY
0,68105c019b,0,1.50,200.0,14.0,50.0,0.016667,2361,167503,0,...,0,0,0,0,0,0,0,0,0,0
1,06d11c4edd,0,0.67,100.0,2.5,30.0,0.000800,1255,1839,0,...,0,0,0,0,0,0,0,0,0,1
2,f6599edba1,0,0.67,64.0,2.8,48.0,0.000625,1138,787,0,...,0,0,0,1,0,0,0,0,0,0
3,0f4bcc2d36,0,1.78,139.0,2.2,4.7,0.010000,1579,2049,0,...,0,0,0,0,0,0,0,0,0,0
4,74db502ed5,0,1.50,400.0,6.4,22.2,0.002381,1050,2982,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12192,18eaca644e,0,0.73,100.0,5.6,300.0,0.002500,1265,2353,0,...,0,0,0,0,0,0,0,0,0,0
12193,eb25cd769e,0,0.72,400.0,5.0,100.0,0.003125,1503,3541,0,...,0,0,0,0,0,0,0,0,0,0
12194,73302461d0,0,1.50,400.0,4.0,21.0,0.000250,2501,14581,0,...,0,0,0,0,0,0,0,0,0,0
12195,f2587b2b08,0,1.50,800.0,1.8,50.0,0.003125,2508,9247,0,...,0,0,0,1,0,0,0,0,0,0


In [29]:
import pickle
model = pickle.load(open("NLP", 'rb'))

In [273]:
df_k=pd.read_csv('/kaggle/input/can-i-be-an-influencer/train_keywords.csv')

In [274]:
# df_k["classification_keyword"].value_counts()

In [275]:
df_k= df_k.loc[df_k.groupby('image_id')['confidence_value'].idxmax()]

In [276]:
df_k["classification_keyword"].value_counts()

nature        2323
plant         1726
outdoors       731
animal         605
water          434
              ... 
gold             1
smoke pipe       1
gift             1
face             1
blackbird        1
Name: classification_keyword, Length: 586, dtype: int64

In [277]:
# import gensim.downloader as api
# from gensim.models import KeyedVectors


In [278]:
# df_k.iloc[[df_k["classification_keyword"].value_counts().head(10).index]]
keywords=df_k["classification_keyword"].value_counts().head(10).index.tolist()

In [279]:
# def find_key(n,r,threshold):
#     l=[df_k['classification_keyword'].iloc[0].lower()]
# #     print(l[0])
#     while(len(l)<=n):
# #         print(threshold)
#         for row in range(1,r):
#             word=df_k['classification_keyword'].iloc[row].lower()
#             try:
#                 model[word]
#                 if word in l:
#                     continue
#                 elif len(l)>0:
#                     new_d={}
#                     for i in l:
#                         new_d[i]=model.similarity(word,i)
#                     max_value = max(new_d, key=new_d.get)
#                     if new_d[max_value]<threshold:
#                         l.append(word)
# #                         print(word)
#                     else:
#                         continue
#             except:
#                 continue
#         threshold+=0.05
#     return l

# keywords=find_key(10,50,0.2)

In [280]:
keywords

['nature',
 'plant',
 'outdoors',
 'animal',
 'water',
 'mammal',
 'tree',
 'person',
 'human',
 'soil']

In [281]:
def reduction(x,keywords,threshold):
    if x in keywords:
        return str(x)
    else:
        d={}
#         d={'plant':0,'outdoors':0,'nature':0,'animal':0,'clothing':0,'apparel':0,'symbol':0,'tree':0,'mammal':0,'building':0,'others':0}
        for i in range(len(keywords)):
            try:
                d[keywords[i]]=model.similarity(x,keywords[i])
            except:
                return 'others'
            max_value = max(d, key=d.get)
            if d[max_value]>threshold:
                return str(max_value)
            else:
                return 'others'
threshold=0.3
df_k["classification_keyword"]=df_k["classification_keyword"].apply(reduction,args=[keywords,threshold])

In [282]:
# df_k

In [283]:
# def drop_dup(df_k):
#     c=0
#     rows_to_drop=[]
#     for id in unique:
# #         print(id)
#         if c%100==0:
#             print(c)
#         tmp=(df_k.loc[df_k['image_id']==id]).sort_values('confidence_value',ascending=False).index.tolist()
# #         print(id,len(tmp))
#         rows_to_drop.append(tmp[1:])
        
# #         df_k = df_k.drop(index=rows_to_drop)
#         c+=1
#     return rows_to_drop

# rows_to_drop=drop_dup(df_k)

In [284]:
# df_k= df_k.loc[df_k.groupby('image_id')['confidence_value'].idxmax()]
# df_k.drop("index",axis=1)

In [286]:
df_k["classification_keyword"].value_counts()

nature      4879
others      2627
plant       1726
outdoors     731
animal       605
water        434
mammal       264
tree         261
person       248
human        231
soil         191
Name: classification_keyword, dtype: int64

In [ ]:
#STOP

In [287]:
df_k=onehot(df_k,"classification_keyword")
df_k

,image_id,image_classification,confidence_value,classification_keyword_animal,classification_keyword_human,classification_keyword_mammal,classification_keyword_nature,classification_keyword_others,classification_keyword_outdoors,classification_keyword_person,classification_keyword_plant,classification_keyword_soil,classification_keyword_tree,classification_keyword_water
724268,000060f5b4,Predicted,99.155289,0,0,0,0,0,1,0,0,0,0,0
595643,00048eb0af,Predicted,99.849998,0,0,0,0,0,0,0,1,0,0,0
437899,000a71e54b,Predicted,99.875671,0,0,0,1,0,0,0,0,0,0,0
920535,000ad1b637,Predicted,99.802711,0,0,0,1,0,0,0,0,0,0,0
921394,00192ccaaa,Predicted,99.993782,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731639,ffe1a4b194,Predicted,99.995323,0,0,0,0,1,0,0,0,0,0,0
880042,ffe4d6ea46,Predicted,98.373611,1,0,0,0,0,0,0,0,0,0,0
727416,ffe7b6ae59,Predicted,87.730011,0,0,0,0,1,0,0,0,0,0,0
856569,ffed1eaaba,Predicted,93.122841,0,0,0,0,0,0,0,0,1,0,0


In [288]:
df_k=labelencoding(df_k,"image_classification")
df_k

,image_id,image_classification,confidence_value,classification_keyword_animal,classification_keyword_human,classification_keyword_mammal,classification_keyword_nature,classification_keyword_others,classification_keyword_outdoors,classification_keyword_person,classification_keyword_plant,classification_keyword_soil,classification_keyword_tree,classification_keyword_water
724268,000060f5b4,0,99.155289,0,0,0,0,0,1,0,0,0,0,0
595643,00048eb0af,0,99.849998,0,0,0,0,0,0,0,1,0,0,0
437899,000a71e54b,0,99.875671,0,0,0,1,0,0,0,0,0,0,0
920535,000ad1b637,0,99.802711,0,0,0,1,0,0,0,0,0,0,0
921394,00192ccaaa,0,99.993782,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731639,ffe1a4b194,0,99.995323,0,0,0,0,1,0,0,0,0,0,0
880042,ffe4d6ea46,0,98.373611,1,0,0,0,0,0,0,0,0,0,0
727416,ffe7b6ae59,0,87.730011,0,0,0,0,1,0,0,0,0,0,0
856569,ffed1eaaba,0,93.122841,0,0,0,0,0,0,0,0,1,0,0


In [289]:
df_k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12197 entries, 724268 to 129352
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   image_id                         12197 non-null  object 
 1   image_classification             12197 non-null  int64  
 2   confidence_value                 12197 non-null  float64
 3   classification_keyword_animal    12197 non-null  uint8  
 4   classification_keyword_human     12197 non-null  uint8  
 5   classification_keyword_mammal    12197 non-null  uint8  
 6   classification_keyword_nature    12197 non-null  uint8  
 7   classification_keyword_others    12197 non-null  uint8  
 8   classification_keyword_outdoors  12197 non-null  uint8  
 9   classification_keyword_person    12197 non-null  uint8  
 10  classification_keyword_plant     12197 non-null  uint8  
 11  classification_keyword_soil      12197 non-null  uint8  
 12  classificati

In [291]:
df_merge = pd.merge(df, df_k, on='image_id', how='inner')
df_merge

,image_id,image_category,aspect_ratio,iso,aperture_value,focal_length,exposure_time,total_days,stats_downloads,camera_make_mod_APPLE,...,classification_keyword_human,classification_keyword_mammal,classification_keyword_nature,classification_keyword_others,classification_keyword_outdoors,classification_keyword_person,classification_keyword_plant,classification_keyword_soil,classification_keyword_tree,classification_keyword_water
0,68105c019b,0,1.50,200.0,14.0,50.0,0.016667,2361,167503,0,...,0,0,0,1,0,0,0,0,0,0
1,06d11c4edd,0,0.67,100.0,2.5,30.0,0.000800,1255,1839,0,...,0,0,0,0,0,0,0,1,0,0
2,f6599edba1,0,0.67,64.0,2.8,48.0,0.000625,1138,787,0,...,0,0,1,0,0,0,0,0,0,0
3,0f4bcc2d36,0,1.78,139.0,2.2,4.7,0.010000,1579,2049,0,...,0,0,0,0,0,0,0,0,0,1
4,74db502ed5,0,1.50,400.0,6.4,22.2,0.002381,1050,2982,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12062,18eaca644e,0,0.73,100.0,5.6,300.0,0.002500,1265,2353,0,...,0,0,1,0,0,0,0,0,0,0
12063,eb25cd769e,0,0.72,400.0,5.0,100.0,0.003125,1503,3541,0,...,0,0,0,0,0,0,0,0,0,0
12064,73302461d0,0,1.50,400.0,4.0,21.0,0.000250,2501,14581,0,...,0,0,0,0,0,0,0,0,0,1
12065,f2587b2b08,0,1.50,800.0,1.8,50.0,0.003125,2508,9247,0,...,0,0,0,0,0,0,1,0,0,0
